In [1]:
#! pip install beautifulsoup4, requests
# !pip install selenium
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from time import sleep
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import numpy as np
import json
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support import ui
import os
import sys
os.path.dirname(sys.executable)
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time

In [47]:
df = pd.read_csv('musea_city_cat.csv')
df.head()

,publicName,city,mainCategory
0,afrika museum,berg en dal,NaN
1,agrarisch museum westerhem,middenbeemster,businesssciencetechnology
2,airborne museum at hartenstein,oosterbeek,history
3,allard pierson,amsterdam,history
4,amsterdam pipe museum,amsterdam,history


In [29]:
'''create headless webdriver'''
options = webdriver.FirefoxOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')

In [48]:
'''lists for iteration'''
data = df['publicName'].tolist()
len(data)
#it = iter(data)
#type(it)
citydata = df['city'].tolist()
catdata = df['mainCategory'].tolist()

In [31]:
data_extract = [data[9], data[36], data[74], data[96], data[104], data[154], data[189], data[221], data[224], data[235], data[248], data[270],
        data[359], data[360], data[363], data[444], data[457], data[458], data[492], data[493], data[495]]

In [32]:
data_city = [citydata[9], citydata[36], citydata[74], citydata[96], citydata[104], citydata[154], citydata[189], citydata[221], citydata[224], citydata[235],
                citydata[248], citydata[270], citydata[359], citydata[360], citydata[363], citydata[444], citydata[457], citydata[458], citydata[492], citydata[493], citydata[495]]

In [33]:
len(data_extract), len(data_city)

(21, 21)

In [34]:
'''functioin to enumerate over search terms'''
enum_extract = enumerate(data_extract)
enum_city = enumerate(data_city)

def getextract():
    key = next(enum_extract)
    return str(key)

def getcity():
    key = next(enum_city)
    return str(key)

In [37]:
'''stripping anything werid from the search term'''
def searchconcat():
    searchfor = getextract() + getcity()
    searchfor = searchfor.replace(',', '')\
    .replace('-','')\
    .replace('(','')\
    .replace(')','')\
    .replace('0', '')\
    .replace('1', '')\
    .replace('2', '')\
    .replace('3', '')\
    .replace('4', '')\
    .replace('5', '')\
    .replace('6', '')\
    .replace('7', '')\
    .replace('8', '')\
    .replace('9', '')\
    .replace(r'[^\w\s]+', '')\
    .strip()
    
    return str(searchfor)

In [38]:
titles = []
headliner = []
searches = []
categories = []
ratings = []
catches = []
names = []
types_ = []
beoordelingen = []

In [39]:
'''Selenoum search automation can either fetch the search item from the list or text can be entered'''
def searchmuseum():
    headlines = []
    #startdriver
    time.sleep(5)
    driver = webdriver.Firefox(options=options) #options=options
    driver.get('https://tripadvisor.nl')
    #get the search html object
    driver.maximize_window()
    time.sleep(5)
    search = driver.find_element_by_css_selector('._3TPJs5_m > form:nth-child(2) > input:nth-child(1)')
    search.send_keys(searchconcat())
    search.submit()
    time.sleep(5)

    #click on first item in results
        #museum = driver.find_element_by_css_selector('div.content-column:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)').click()
        #museum = driver.find_element_by_css_selector('div.content-column:nth-child(1)').click()
    
    try: #museum = driver.find_element_by_css_selector('div.content-column:nth-child(1) > div:nth-child(1) > div:nth-child(1)').click() 
        #driver.find_element_by_css_selector('div.search-filters > ul > li:nth-child(5) > a').click()
        museum = driver.find_element_by_css_selector('div.result-title').click()
        
    #getting the headline
        time.sleep(5)
        driver.maximize_window()
        head = driver.find_element_by_css_selector('div.is-12-tablet')
        headline = head.text.strip()
        headline_lst =list(headline.split("\n"))
        type_ = headline_lst[1] 
        name = headline_lst[2]
        beoordeling = headline_lst[3]
        
     #safe the title of the first object
        #title = driver.find_element_by_id('HEADING')
        #title = driver.title()
    #getting source for beautiful soup
        source = driver.page_source
        soup = BeautifulSoup(source, 'html.parser')
        soup_list = list(soup.stripped_strings)
        catch = soup_list[15:25] #catch all
        category = soup_list[18:19] #category
        title = soup_list[20] #name of museum
        rating = soup_list[21] #rating
        
    
        #soup = soup.getText(',').strip()

        #for string in soup.stripped_strings:
         #   soup_out = repr(string)
        
        driver.quit()  
        
        #for _ in str(soup_out):
         #   soup_outs.append(_)
            
        return type_, name, beoordeling, catch, category, rating, title  
    except (NoSuchElementException, NameError):
        print('element error!')
        driver.quit()
        no_element = 'no'
        return no_element, no_element, no_element, no_element, no_element, no_element, no_element

In [40]:
for i in range(21):
    type_, name, beoordeling, catch, category, rating, title  = searchmuseum()
    names += [name]
    types_ += [type_]
    beoordelingen += [beoordeling]
    catches += [catch]
    categories += [category]
    ratings += [rating] 
    titles += [title]
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
element error!
12
13
14
15
16
17
18
19
element error!
20


In [41]:
df_3 = pd.DataFrame(list(zip(data_extract, names, titles, types_, beoordelingen, ratings, catches )),
                      columns = ['publicName', 'title', 'title2', 'category', 'rating', 'rating2', 'summary'])

df_3.to_csv('scrap.v2.csv', encoding='utf-8', index=False)

In [44]:
df_4 = df_3.copy()
#data_extract = [data[9], data[36], data[74], data[96], data[104], data[154], data[189], data[221], data[224], data[235], data[248], data[270],
        #data[359], data[360], data[363], data[444], data[457], data[458], data[492], data[493], data[495]]

In [45]:
df_4.drop([0, 1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 16, 18, 19, 20])

,publicName,title,title2,category,rating,rating2,summary
4,humanity house,Humanity House,Humanity House,Kunstmusea,52 beoordelingen,52 beoordelingen,[voor reisupdates en om berichten naar andere ...
5,museum more,Museum MORE,Museum MORE,Kunstmusea,373 beoordelingen,373 beoordelingen,[voor reisupdates en om berichten naar andere ...
13,centraal museum,Centraal Museum,Centraal Museum,Kunstmusea,398 beoordelingen,398 beoordelingen,[voor reisupdates en om berichten naar andere ...
14,de oude kerk,Oude Kerk,Oude Kerk,Religieuze locaties,1.094 beoordelingen,1.094 beoordelingen,[voor reisupdates en om berichten naar andere ...
15,touwmuseum de baanschuur,'Het Touwmuseum'- Oudewater,'Het Touwmuseum'- Oudewater,Specialiteitenmusea,15 beoordelingen,15 beoordelingen,[voor reisupdates en om berichten naar andere ...
17,vincent van goghhuis,Vincent van Gogh Huis,Vincent van Gogh Huis,Kunstmusea,57 beoordelingen,57 beoordelingen,[voor reisupdates en om berichten naar andere ...


In [46]:
df_4.to_csv('scrap.v4.csv', encoding='utf-8', index=False)

In [45]:
data_extract

['bak basis voor actuele kunst',
 'de halve maen',
 'haags historisch museum',
 'het palthe huis',
 'humanity house',
 'museum more',
 'marechausseemuseum',
 'museum valkenburg',
 'huis van het boek',
 'museum rotterdam',
 'museum van de 20e eeuw',
 'nationaal gevangenismuseum',
 'bureau europa',
 'centraal museum',
 'de oude kerk',
 'touwmuseum de baanschuur',
 'libertum',
 'vincent van goghhuis',
 'museumbrouwerij de roos',
 'beeld en geluid den haag',
 'portrait gallery of the 17th century']

In [49]:
data_search = [data[6], data[9], data[29], data[36], data[74], data[96], data[110], data[131], data[189], data[221], data[224], data[235],
              data[248], data[265], data[270], data[312], data[316], data[358], data[359], data[367], data[371], data[398], data[457], data[492],
              data[493], data[495]]

In [50]:
enum_search = enumerate(data_search)

In [52]:
def getsearch():
    key = next(enum_search)
    return str(key)

In [77]:
def searchconcat():
    searchfor = k + ''
    searchfor = searchfor.replace(',', '')\
    .replace('-','')\
    .replace('(','')\
    .replace(')','')\
    .replace('0', '')\
    .replace('1', '')\
    .replace('2', '')\
    .replace('3', '')\
    .replace('4', '')\
    .replace('5', '')\
    .replace('6', '')\
    .replace('7', '')\
    .replace('8', '')\
    .replace('9', '')\
    .replace(r'[^\w\s]+', '')\
    .strip()
    
    return str(searchfor)

In [200]:
#Search for museum
def searchmuseum():
    headlines = []
    #startdriver
    time.sleep(5)
    driver = webdriver.Firefox() #options=options
    driver.get('https://tripadvisor.nl')
    #get the search html object
    driver.maximize_window()
    time.sleep(5)
    search = driver.find_element_by_css_selector('._3TPJs5_m > form:nth-child(2) > input:nth-child(1)')
    search.send_keys('museum voor communicatie den haag')
    search.submit()
    time.sleep(5)

    #click on first item in results
        #museum = driver.find_element_by_css_selector('div.content-column:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)').click()
        #museum = driver.find_element_by_css_selector('div.content-column:nth-child(1)').click()
    
    try: #museum = driver.find_element_by_css_selector('div.content-column:nth-child(1) > div:nth-child(1) > div:nth-child(1)').click() 
        #driver.find_element_by_css_selector('div.search-filters > ul > li:nth-child(5) > a').click()
        museum = driver.find_element_by_css_selector('div.result-title').click()
        
    #getting the headline
        time.sleep(5)
        driver.maximize_window()
        head = driver.find_element_by_css_selector('div.is-12-tablet')
        headline = head.text.strip()
        headline_lst =list(headline.split("\n"))
        type_ = headline_lst[1] 
        name = headline_lst[2]
        beoordeling = headline_lst[3]
        
     #safe the title of the first object
        #title = driver.find_element_by_id('HEADING')
        #title = driver.title()
    #getting source for beautiful soup
        source = driver.page_source
        soup = BeautifulSoup(source, 'html.parser')
        soup_list = list(soup.stripped_strings)
        catch = soup_list[15:25] #catch all
        category = soup_list[18:19] #category
        title = soup_list[20] #name of museum
        rating = soup_list[21] #rating
        
    
        #soup = soup.getText(',').strip()

        #for string in soup.stripped_strings:
         #   soup_out = repr(string)
        
        driver.quit()  
        
        #for _ in str(soup_out):
         #   soup_outs.append(_)
            
        return type_, name, beoordeling, catch, category, rating, title  
    except (NoSuchElementException, NameError):
        print('element error!')
        driver.quit()
        no_element = 'no'
        return no_element, no_element, no_element, no_element, no_element, no_element, no_element


In [201]:
type_, name, beoordeling, catch, category, rating, title  = searchmuseum()
type_, name, beoordeling, catch, category, rating, title

('Kindermusea',
 'Museum voor Communicatie',
 '23 beoordelingen',
 ['voor reisupdates en om berichten naar andere reizigers te sturen.',
  "Zoekresultaten die overeenkomen met '",
  'museum voor communicatie den haag',
  "'",
  'Kindermusea',
  'Museum voor Communicatie',
  '23 beoordelingen',
  'Zeestraat 82, Den Haag, Zuid-Holland, Nederland',
  'Zeestraat 82, Den Haag',
  'Locaties'],
 ["'"],
 '23 beoordelingen',
 'Museum voor Communicatie')

In [202]:
for i in range(1):
    names += [name]
    types_ += [type_]
    beoordelingen += [beoordeling]
    catches += [catch]
    categories += [category]
    ratings += [rating] 
    titles += [title]
    print(i)

0


In [203]:
data_search = [data[6], data[74], data[96], data[110], data[131], data[131], data[189], data[221], data[224],
               data[270], data[312], data[316], data[359], data[371], data[398], data[495], data[493]]
data_search

['anton pieck museum',
 'haags historisch museum',
 'het palthe huis',
 'joods historisch museum',
 'koninklijk paleis amsterdam',
 'koninklijk paleis amsterdam',
 'marechausseemuseum',
 'museum valkenburg',
 'huis van het boek',
 'nationaal gevangenismuseum',
 'oudheidkamer texel',
 'panorama mesdag',
 'bureau europa',
 'kunstmuseum den haag',
 'museum menkemaborg',
 'portrait gallery of the 17th century',
 'beeld en geluid den haag']

In [204]:
df_5 = pd.DataFrame(list(zip(data_search, names, titles, types_, beoordelingen, ratings, catches )),
                      columns = ['publicName', 'title', 'title2', 'category', 'rating', 'rating2', 'summary'])
df_5

,publicName,title,title2,category,rating,rating2,summary
0,anton pieck museum,Anton Pieck Museum,Anton Pieck Museum,Kunstmusea,83 beoordelingen,83 beoordelingen,[voor reisupdates en om berichten naar andere ...
1,haags historisch museum,Haags Historisch Museum,Haags Historisch Museum,Kunstmusea,170 beoordelingen,170 beoordelingen,[voor reisupdates en om berichten naar andere ...
2,het palthe huis,Palthehuis Museum,Palthehuis Museum,Specialiteitenmusea,9 beoordelingen,9 beoordelingen,[voor reisupdates en om berichten naar andere ...
3,joods historisch museum,Joods Historisch Museum,Joods Historisch Museum,Historische musea,1.045 beoordelingen,1.045 beoordelingen,[voor reisupdates en om berichten naar andere ...
4,koninklijk paleis amsterdam,Beste resultaat voor 'royal palace',Beste resultaat voor ',"Zoek in plaats daarvan naar ""royale palace ams...",Architectonische bezienswaardigheden,royal palace,[voor reisupdates en om berichten naar andere ...
5,koninklijk paleis amsterdam,Royal Palace Amsterdam,Royal Palace Amsterdam,Architectonische bezienswaardigheden,3.332 beoordelingen,3.332 beoordelingen,[voor reisupdates en om berichten naar andere ...
6,marechausseemuseum,Museum Marechaussee,Museum Marechaussee,Historische musea,24 beoordelingen,24 beoordelingen,[voor reisupdates en om berichten naar andere ...
7,museum valkenburg,Museum Valkenburg,Museum Valkenburg,Kunstmusea,35 beoordelingen,35 beoordelingen,[voor reisupdates en om berichten naar andere ...
8,huis van het boek,Museum Meermanno,Museum Meermanno,Architectonische bezienswaardigheden,67 beoordelingen,67 beoordelingen,[voor reisupdates en om berichten naar andere ...
9,nationaal gevangenismuseum,Gevangenismuseum,Gevangenismuseum,Historische musea,11 beoordelingen,11 beoordelingen,[voor reisupdates en om berichten naar andere ...


In [205]:
df_5.to_csv('scrap.v5.csv', encoding='utf-8', index=False)

In [89]:
#df_scrap = pd.read_csv('scrap.v1.csv')
df_scrap[300:320]

,publicName,title,title2,category,rating,rating2,summary
300,nederlands zilvermuseum,Zilvermuseum,Zilvermuseum,Specialiteitenmusea,80 beoordelingen,80 beoordelingen,['voor reisupdates en om berichten naar andere...
301,nederlands zouavenmuseum,Stichting Nederlands Zouavenmuseum,Stichting Nederlands Zouavenmuseum,Specialiteitenmusea,13 beoordelingen,13 beoordelingen,['voor reisupdates en om berichten naar andere...
302,nemo science museum,NEMO Science Museum,NEMO Science Museum,Kindermusea,4.325 beoordelingen,4.325 beoordelingen,['voor reisupdates en om berichten naar andere...
303,nijntje museum,Nijntje Museum,Nijntje Museum,Specialiteitenmusea,395 beoordelingen,395 beoordelingen,['voor reisupdates en om berichten naar andere...
304,noordelijk scheepvaartmuseum,Noordelijk Scheepvaartmuseum,Noordelijk Scheepvaartmuseum,Specialiteitenmusea,155 beoordelingen,155 beoordelingen,['voor reisupdates en om berichten naar andere...
305,noordveluws museum,Neka Museum,Neka Museum,Specialiteitenmusea,997 beoordelingen,997 beoordelingen,['voor reisupdates en om berichten naar andere...
306,observeum burgum,"Friesland, Nederland","Friesland, Nederland",Burgum,122 beoordelingen en meningen,122 beoordelingen en meningen,['voor reisupdates en om berichten naar andere...
307,oorlogsmuseum overloon,Oorlogsmuseum,Oorlogsmuseum,Militaire musea,575 beoordelingen,575 beoordelingen,['voor reisupdates en om berichten naar andere...
308,openbaar vervoer speelgoed museum doetinchem,Openbaar Vervoer & Speelgoed Museum,Openbaar Vervoer & Speelgoed Museum,Specialiteitenmusea,8 beoordelingen,8 beoordelingen,['voor reisupdates en om berichten naar andere...
309,openluchtmuseum en themapark de spitkeet,no,no,no,no,no,no


In [193]:
#changing specific columns
df_scrap.iloc[492]

publicName                              museumbrouwerij de roos
title                                     Bierbrouwerij de Roos
title2                                                   Parijs
category                                            Brouwerijen
rating                                           29 beoordeling
rating2                                Ile-de-France, Frankrijk
summary       ['voor reisupdates en om berichten naar andere...
Name: 492, dtype: object

In [192]:
'''manual annotation of some of the museums which could not be fetched'''
df_scrap.at[492, 'title'] = 'Bierbrouwerij de Roos'
df_scrap.at[492, 'rating'] = '29 beoordeling'
df_scrap.at[492, 'category'] = 'Brouwerijen'

In [14]:

#df_5.drop([4])

,publicName,title,title2,category,rating,rating2,summary
0,anton pieck museum,Anton Pieck Museum,Anton Pieck Museum,Kunstmusea,83 beoordelingen,83 beoordelingen,['voor reisupdates en om berichten naar andere...
1,haags historisch museum,Haags Historisch Museum,Haags Historisch Museum,Kunstmusea,170 beoordelingen,170 beoordelingen,['voor reisupdates en om berichten naar andere...
2,het palthe huis,Palthehuis Museum,Palthehuis Museum,Specialiteitenmusea,9 beoordelingen,9 beoordelingen,['voor reisupdates en om berichten naar andere...
3,joods historisch museum,Joods Historisch Museum,Joods Historisch Museum,Historische musea,1.045 beoordelingen,1.045 beoordelingen,['voor reisupdates en om berichten naar andere...
5,koninklijk paleis amsterdam,Royal Palace Amsterdam,Royal Palace Amsterdam,Architectonische bezienswaardigheden,3.332 beoordelingen,3.332 beoordelingen,['voor reisupdates en om berichten naar andere...
6,marechausseemuseum,Museum Marechaussee,Museum Marechaussee,Historische musea,24 beoordelingen,24 beoordelingen,['voor reisupdates en om berichten naar andere...
7,museum valkenburg,Museum Valkenburg,Museum Valkenburg,Kunstmusea,35 beoordelingen,35 beoordelingen,['voor reisupdates en om berichten naar andere...
8,huis van het boek,Museum Meermanno,Museum Meermanno,Architectonische bezienswaardigheden,67 beoordelingen,67 beoordelingen,['voor reisupdates en om berichten naar andere...
9,nationaal gevangenismuseum,Gevangenismuseum,Gevangenismuseum,Historische musea,11 beoordelingen,11 beoordelingen,['voor reisupdates en om berichten naar andere...
10,oudheidkamer texel,Oudheidkamer Texel,Oudheidkamer Texel,Historische musea,5 beoordelingen,5 beoordelingen,['voor reisupdates en om berichten naar andere...


In [194]:
df_scrap.to_csv('scrap.v1.csv', encoding='utf-8', index=False)

In [196]:
#Loading the dataframes
df_scrap = pd.read_csv('scrap.v1.csv')
df_5 = pd.read_csv('scrap.v5.csv')
df_4 = pd.read_csv('scrap.v4.csv')
df_3 = pd.read_csv('scrap.v3.csv')

In [277]:
df_2.to_csv('scrap.v6.csv', encoding='utf-8', index=False)

In [239]:
#df_2 = pd.read_csv('scrap.v6.csv')

In [274]:
#merging subframes
#frames = [df_5, df_4, df_3]
#df_2 =pd.concat(frames)
#df_2

In [273]:
#duplicateRow = df_2[df_2.duplicated(['publicName'])]
#duplicateRow

In [276]:
df_2.drop([87], inplace=True)
df_2[30:]

,publicName,title,title2,category,rating,rating2,summary
46,graafs museum,Graafs Museum,Graafs Museum,Historische musea,9 beoordelingen,9 beoordelingen,['voor reisupdates en om berichten naar andere...
48,hunebedcentrum,Hunebed Centrum,Hunebed Centrum,Historische locaties,315 beoordelingen,315 beoordelingen,['voor reisupdates en om berichten naar andere...
50,kasteel doorwerth,Kasteel Doorwerth,Kasteel Doorwerth,Kastelen,286 beoordelingen,286 beoordelingen,['voor reisupdates en om berichten naar andere...
51,kasteel hoensbroek,Kasteel Hoensbroek,Kasteel Hoensbroek,Kastelen,307 beoordelingen,307 beoordelingen,['voor reisupdates en om berichten naar andere...
52,katwijks museum,Katwijks Museum,Katwijks Museum,Kunstmusea,79 beoordelingen,79 beoordelingen,['voor reisupdates en om berichten naar andere...
54,literatuurmuseum,Koninklijke Bibliotheek,Koninklijke Bibliotheek,Bibliotheken,16 beoordelingen,16 beoordelingen,['voor reisupdates en om berichten naar andere...
55,mij museum ijsselstein,Museum Ijsselstein,Museum Ijsselstein,Kunstmusea,20 beoordelingen,20 beoordelingen,['voor reisupdates en om berichten naar andere...
56,museum broeker veiling,Museum BroekerVeiling,Museum BroekerVeiling,Specialiteitenmusea,145 beoordelingen,145 beoordelingen,['voor reisupdates en om berichten naar andere...
59,museum de voorde,1 beoordeling,1 beoordeling,Wetenschapsmusea,1 beoordeling,"Zuidwaarts 2, Zoetermeer, Zuid-Holland, Nederland",['voor reisupdates en om berichten naar andere...
60,museum elburg,Museum Elburg,Museum Elburg,Kunstmusea,27 beoordelingen,27 beoordelingen,['voor reisupdates en om berichten naar andere...


In [278]:
newmuse =df_2['publicName'].tolist()

In [279]:
len(newmuse)

65

In [280]:
#dropping duplicates
df_1 = df_scrap[~df_scrap['publicName'].isin(newmuse)]

In [281]:
len(df_1)

431

In [282]:
#merging the final frames
frames = [df_1, df_2]
df_f = pd.concat(frames)
df_f

,publicName,title,title2,category,rating,rating2,summary
0,afrika museum,Afrika Museum,Afrika Museum,Specialiteitenmusea,174 beoordelingen,174 beoordelingen,['voor reisupdates en om berichten naar andere...
1,agrarisch museum westerhem,Agrarisch Museum Westerhem,Agrarisch Museum Westerhem,Historische musea,1 beoordeling,1 beoordeling,['voor reisupdates en om berichten naar andere...
2,airborne museum at hartenstein,Airborne Museum at Hartenstein,Airborne Museum at Hartenstein,Specialiteitenmusea,899 beoordelingen,899 beoordelingen,['voor reisupdates en om berichten naar andere...
3,allard pierson,Allard Pierson,Allard Pierson,Historische musea,167 beoordelingen,167 beoordelingen,['voor reisupdates en om berichten naar andere...
4,amsterdam pipe museum,Amsterdam Pipe Museum,Amsterdam Pipe Museum,Specialiteitenmusea,326 beoordelingen,326 beoordelingen,['voor reisupdates en om berichten naar andere...
...,...,...,...,...,...,...,...
90,tassenmuseum amsterdam,Tassenmuseum Amsterdam,Tassenmuseum Amsterdam,Specialiteitenmusea,1.758 beoordelingen,1.758 beoordelingen,['voor reisupdates en om berichten naar andere...
91,tropenmuseum,Tropenmuseum,Tropenmuseum,Historische musea,1.128 beoordelingen,1.128 beoordelingen,['voor reisupdates en om berichten naar andere...
92,van gogh museum,Van Gogh Museum,Van Gogh Museum,Kunstmusea,63.786 beoordelingen,63.786 beoordelingen,['voor reisupdates en om berichten naar andere...
93,verzetsmuseum amsterdam,Verzetsmuseum Amsterdam,Verzetsmuseum Amsterdam,Historische musea,3.403 beoordelingen,3.403 beoordelingen,['voor reisupdates en om berichten naar andere...


In [291]:
'''exporting to csv'''
df_f.to_csv('scrap.v.csv', encoding='utf-8', index=False)

In [284]:
new_lst = df_f['publicName'].unique()
len(new_lst)

496

<bound method DataFrame.set_index of                         publicName                           title  \
0                    afrika museum                   Afrika Museum   
1       agrarisch museum westerhem      Agrarisch Museum Westerhem   
2   airborne museum at hartenstein  Airborne Museum at Hartenstein   
3                   allard pierson                  Allard Pierson   
4            amsterdam pipe museum           Amsterdam Pipe Museum   
..                             ...                             ...   
52          tassenmuseum amsterdam          Tassenmuseum Amsterdam   
53                    tropenmuseum                    Tropenmuseum   
54                 van gogh museum                 Van Gogh Museum   
55         verzetsmuseum amsterdam         Verzetsmuseum Amsterdam   
56               zandvoorts museum               Zandvoorts Museum   

                            title2             category                rating  \
0                    Afrika Museum  Speci

In [285]:
duplicateRowsDF = df_f[df_f.duplicated(['publicName'])]
duplicateRowsDF

,publicName,title,title2,category,rating,rating2,summary


In [289]:
del df_f['rating2']
del df_f['title2']

In [290]:
df_f

,publicName,title,category,rating,summary
0,afrika museum,Afrika Museum,Specialiteitenmusea,174 beoordelingen,['voor reisupdates en om berichten naar andere...
1,agrarisch museum westerhem,Agrarisch Museum Westerhem,Historische musea,1 beoordeling,['voor reisupdates en om berichten naar andere...
2,airborne museum at hartenstein,Airborne Museum at Hartenstein,Specialiteitenmusea,899 beoordelingen,['voor reisupdates en om berichten naar andere...
3,allard pierson,Allard Pierson,Historische musea,167 beoordelingen,['voor reisupdates en om berichten naar andere...
4,amsterdam pipe museum,Amsterdam Pipe Museum,Specialiteitenmusea,326 beoordelingen,['voor reisupdates en om berichten naar andere...
...,...,...,...,...,...
90,tassenmuseum amsterdam,Tassenmuseum Amsterdam,Specialiteitenmusea,1.758 beoordelingen,['voor reisupdates en om berichten naar andere...
91,tropenmuseum,Tropenmuseum,Historische musea,1.128 beoordelingen,['voor reisupdates en om berichten naar andere...
92,van gogh museum,Van Gogh Museum,Kunstmusea,63.786 beoordelingen,['voor reisupdates en om berichten naar andere...
93,verzetsmuseum amsterdam,Verzetsmuseum Amsterdam,Historische musea,3.403 beoordelingen,['voor reisupdates en om berichten naar andere...
